In [5]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


This is the main Sensor Kennel application that will 
identify, read and write any sensor that's plugged in.

On the lower side:  
  Pin1 (left) Gnd, Pin3 3v3, P4 (Pin-3 from right) SDA, P15 (Pin-5 from right) SCL.
  P17 (Pin-2) TX, P13 (Upper side Pin-2) RX 
  For BNO055 with PS1=High SCL=TX(17), SDA=RX(13)
  
Do we want to draw a graph of numbers if a button is pressed?

Should we have MQTT numbers going out from it?

Or just keep it comprehensive

Sensors known:
* VL53L0X lidar
* VL6180 lidar
* SI7021 humidity
* TSL561 luminous
* TMP102 temp
* SDOF GyAccMag
* BME280 barhumid
* SHT31D tmphumid
* MLX90621 16x4-ir
* PX4PITOT airspeed
* BNO055 orient
* GPS serial


In [37]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [136]:
%sendtofile --binary --source OLED_driver.py
%sendtofile --quiet --source device_detection.py
%sendtofile --quiet --source SI7021_funcs.py
%sendtofile --quiet --source TSL561_funcs.py
%sendtofile --quiet --source VL53L0X_funcs.py
%sendtofile --quiet --source VL6180_funcs.py
%sendtofile --quiet --source SDOF_funcs.py
%sendtofile --quiet --source SHT31D_funcs.py
%sendtofile --quiet --source BME280_funcs.py
%sendtofile --quiet --source MLX90621_funcs.py
%sendtofile --quiet --source BNO055serial_funcs.py
%sendtofile --quiet --source OLED_grapher.py


Sent 3368 bytes in 113 chunks to OLED_driver.py.
Sent 94 lines (2820 bytes) to device_detection.py.
Sent 55 lines (2040 bytes) to SI7021_funcs.py.
Sent 54 lines (1757 bytes) to TSL561_funcs.py.
Sent 33 lines (975 bytes) to VL53L0X_funcs.py.
Sent 106 lines (3190 bytes) to VL6180_funcs.py.
Sent 42 lines (1800 bytes) to SDOF_funcs.py.
Sent 43 lines (1090 bytes) to SHT31D_funcs.py.
Sent 76 lines (3165 bytes) to BME280_funcs.py.
Sent 170 lines (5915 bytes) to MLX90621_funcs.py.
Sent 87 lines (2701 bytes) to BNO055serial_funcs.py.
Sent 145 lines (4083 bytes) to OLED_grapher.py.


In [17]:
# This requires you to checkout https://github.com/micropython/micropython-lib and avoids the upip
%sendtofile --quiet --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio/uasyncio/__init__.py lib/uasyncio/
%sendtofile --quiet --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.core/uasyncio/core.py lib/uasyncio/
%sendtofile --quiet --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.queues/uasyncio/queues.py lib/uasyncio/
%sendtofile --quiet --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.synchro/uasyncio/synchro.py lib/uasyncio/

# This requires you to checkout https://github.com/peterhinch/micropython-async
%sendtofile --quiet --source /home/julian/extrepositories/micropython-async/asyn.py
%sendtofile --quiet --source /home/julian/extrepositories/micropython-async/aswitch.py

# This requires you to checkout https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as
%sendtofile --quiet --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py

# This is the wrapper code on the asynchronous mqtt
%sendtofile --quiet --source fmqttqueue.py

Sent 638 lines (23042 bytes) to mqtt_as.py.


In [131]:
%sendtofile config.txt

wifiname     mqtt-wifi
wifipassword decafbad00
mqttbroker   10.3.141.1
pinled       25
boardname    heltec1


Sent 5 lines (108 bytes) to config.txt.


In [55]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [122]:
aloop = asyncio.get_event_loop()
aloop.create_task(fqm.fstreamtask())


In [128]:
fqm.addsubscription("qqq")
async def sm(fqm):
    for i in range(20):
        fqm.fput("jjj", 'TT%d'%i)
        await asyncio.sleep_ms(1000)
        print(i)
        while fqm.rqueue:
            print("rqueue", fqm.rqueue.popleft())
aloop.run_until_complete(sm(fqm))


DRerunning conn_han qqq
DRunning conn_han 1 2
.0
rqueue (b'qqq', b'1552785000492')
1
2
3
rqueue (b'ggg', b'1552785003300')
4
.5
6
7
rqueue (b'qqq', b'1552785006883')
8
9
RAM free 90688 alloc 36736
10
11
.12
13
14
15
16
.17
18
19


In [104]:
async def kk():
    print("kk")
    await fqm.client.publish("jj", "ll", qos=1)
    print("kk")
aloop.run_until_complete(kk())


kk
kk


In [71]:
print(fqm.client.isconnected())
fqm.client.publish("ting/thing", "ffff")

True


In [33]:
fqm.fput('TT%d'%9999)
print(len(fqm._queue))

0


In [2]:
%serialconnect


serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [36]:
print(dir(uasyncio))

['__class__', '__file__', '__name__', '__path__', 'DEBUG', 'TimeoutError', 'log', 'select', 'sleep', 'sleep_ms', 'time', 'ucollections', 'uerrno', 'utimeq', 'uasyncio', 'get_event_loop', '_socket', 'core', 'set_debug', 'PollEventLoop', 'EventLoop', 'StreamReader', 'IORead', 'IOReadDone', 'StreamWriter', 'IOWrite', 'IOWriteDone', 'open_connection', 'start_server', 'type_gen', 'CancelledError', 'SysCall1', 'SleepMs', 'StopLoop', 'SysCall', 'cancel', 'TimeoutObj', 'wait_for_ms', 'wait_for', 'coroutine', 'ensure_future', 'Task']


In [18]:
aloop = asyncio.get_event_loop()
aloop.run_until_complete(sm())


0
1
2


*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
  File "/lib/uasyncio/core.py", line 180, in run_until_complete
  File "/lib/uasyncio/core.py", line 173, in run_forever
  File "/lib/uasyncio/__init__.py", line 69, in wait
KeyboardInterrupt: 


In [138]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [ ]:
#%sendtofile main.py

# connect to the OLED and initialize (and get its i2c bus)
from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext
import machine, time

from device_detection import IdentifyI2CDevice, IdentifyUARTDevice

from machine import Pin
import uasyncio as asyncio
from fmqttqueue import FMQTTQueue

fconfig = dict(x.split()  for x in open("config.txt"))
pled = Pin(int(fconfig["pinled"]), Pin.OUT)

devices = IdentifyI2CDevice(i2c)
if not devices:
    uart = machine.UART(1, baudrate=9600, rx=13, tx=17)
    # gps is 9600, bluefly is 57600  
    devices = IdentifyUARTDevice(uart)
if not devices:
    import onewire, ds18x20
    pindallas = machine.Pin(13, Pin.OUT, Pin.PULL_UP)
    dallasobj = ds18x20.DS18X20(onewire.OneWire(pindallas))
    dallasscanned = dallasobj.scan()
    if dallasscanned:
        sdallasqorder = {347:0, 376:1, 448:2, 475:3, 576:4, 311:5, 406:6, 157:7}
        dallasscanned.sort(key=lambda X: sdallasqorder.get(sum(X), 0))
        devices = ["dallas n=%d"%len(dallasscanned)]
        
if not devices:
    devices.append("nothing found")

# this might look like a lot of code, but it's the complete 
# visual output of text and numbers that we do per sensor here
for dev in devices:
    devname, devdesc = dev.split(" ", 1)
    fbuff.fill(0)
    fbuff.text(devname, 0, 0, 1)
    fbuff.text(devdesc, 0, 12, 1)
    doublepixels()
    oledshow()

async def plotdevices(fqm):
    topic = fconfig["boardname"]+"/"+devname
    if devname == "VL53L0X":
        from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist
        VL53L0Xinit(i2c)
        while True:
            d = VL53L0Xdist()
            fbuff.fill(0)
            fbuff.text("VL53L0X", 0, 0, 1)
            fbuff.text("dist(mm):", 8, 20, 1)
            fatntext("%d" % d, 40, 36)
            oledshow()
            fqm.fput(topic, "%d"%d)
            await asyncio.sleep_ms(50)
        
    if devname == "VL6180":
        from VL6180_funcs import VL6180init, distmm
        VL6180init(i2c)
        while True:
            d = distmm()
            fbuff.fill(0)
            fbuff.text("VL6180", 0, 0, 1)
            fbuff.text("dist(mm):", 8, 20, 1)
            fatntext("%d" % d, 40, 36)
            oledshow()
            fqm.fput(topic, "%d" % d)
            await asyncio.sleep_ms(50)


# setup and operation of the async loop
fqm = FMQTTQueue(pled)
fqm.setupmqttas(fconfig["wifiname"], fconfig["wifipassword"], fconfig["mqttbroker"])
fqm.addsubscription(fconfig["boardname"]+"/msg")

aloop = asyncio.get_event_loop()
aloop.create_task(fqm.fstreamtask())
aloop.run_until_complete(plotdevices(fqm))


Revision ID: 0x0 Device ID: 0x1
revision (1, 3) module_revision (1, 2)
DConnecting.
.Checking WiFi integrity.
.Got reliable connection
Connecting to broker.
Connected to broker.
DConnected.
DRunning conn_han None 1
....RAM free 82368 alloc 45056
....RAM free 82320 alloc 45104
....RAM free 82496 alloc 44928
..

In [ ]:
if devname == "dallas":
    while True:
        dallasobj.convert_temp()
        time.sleep(0.8)
        fbuff.fill(0)
        fbuff.text("dallas temps", 0, 0, 1)
        for i, d in enumerate(dallasscanned):
            try:
                t = dallasobj.read_temp(d)
            except Exception:
                t = -100
            fbuff.text("%.2f"%t, (i%2)*64, (i//2+2)*8)
            if mqttclient is not None and t != -100:
                mqttclient.publish(b"sk1/dallas/%d"%i, b"%.2f"%t)
        if mqttclient:
            fbuff.text("**MQTT", 0, 56)
        oledshow()
    
if devname == "SI7021":
    from SI7021_funcs import setupSI7021, SI7021humiditytemp
    setupSI7021(i2c)
    while True:
        h, t = SI7021humiditytemp()
        fbuff.fill(0)
        fbuff.text("SI7021", 0, 0, 1)
        fbuff.text("humid:", 0, 23, 1)
        fatntext("%.1f"%h, 50, 16)
        fbuff.text("temp:", 0, 53, 1)
        fatntext("%.1f"%t, 50, 46)
        if mqttclient is not None:
            mqttclient.publish(b"sk1/humidity", b"%.1f"%h)
            mqttclient.publish(b"sk1/temperature", b"%.1f"%t)
            fbuff.text("**MQTT", 0, 32)

        oledshow()
        time.sleep(0.25)

if devname == "VL53L0X":
    from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist
    VL53L0Xinit(i2c)
    while True:
        d = VL53L0Xdist()
        fbuff.fill(0)
        fbuff.text("VL53L0X", 0, 0, 1)
        fbuff.text("dist(mm):", 8, 20, 1)
        fatntext("%d" % d, 40, 36)
        oledshow()
        
if devname == "VL6180":
    from VL6180_funcs import VL6180init, distmm
    VL6180init(i2c)
    while True:
        d = distmm()
        fbuff.fill(0)
        fbuff.text("VL53L0X", 0, 0, 1)
        fbuff.text("dist(mm):", 8, 20, 1)
        fatntext("%d" % d, 40, 36)
        oledshow()

if devname == "TSL561":
    from TSL561_funcs import setupTSL561, luminosity, setregtimings
    setupTSL561(i2c)
    setregtimings(True, 1)
    while True:
        br, ir = luminosity()
        fbuff.fill(0)
        fbuff.text("TSL561", 0, 0, 1)
        fbuff.text("broad:", 0, 23, 1)
        fatntext("%d"%br, 50, 16)
        fbuff.text("ir:", 0, 53, 1)
        fatntext("%d"%ir, 50, 46)
        oledshow()
        time.sleep(0.1)

if devname == "TMP102":
    while True:
        r = i2c.readfrom_mem(0x48, 0x00, 2)
        t = r[0] + r[1]/256
        fbuff.fill(0)
        fbuff.text("TMP102", 0, 0, 1)
        fbuff.text("temp:", 0, 23, 1)
        fatntext("%.03f"%t, 30, 40)
        oledshow()
        time.sleep(0.1)

if devname == "SDOF":
    from SDOF_funcs import SetupAccGyrMag, readvectorsensor
    SetupAccGyrMag(i2c)
    while True:
        fbuff.fill(0)
        x, y, z = readvectorsensor("a")
        fbuff.text("Acc:", 0, 0, 1)
        fbuff.text("%d"%x, 8, 8, 1)
        fbuff.text("%d"%y, 8, 16, 1)
        fbuff.text("%d"%z, 8, 24, 1)
        x, y, z = readvectorsensor("g")
        fbuff.text("gyr:", 64, 0, 1)
        fbuff.text("%d"%x, 72, 8, 1)
        fbuff.text("%d"%y, 72, 16, 1)
        fbuff.text("%d"%z, 72, 24, 1)
        x, y, z = readvectorsensor("c")
        fbuff.text("mag:", 40, 32, 1)
        fbuff.text("%d"%x, 48, 40, 1)
        fbuff.text("%d"%y, 48, 48, 1)
        fbuff.text("%d"%z, 48, 56, 1)
        oledshow()
        time.sleep(0.1)
        
if devname == "BME280" or devname == "BME180":
    from BME280_funcs import bme280init, readBME280
    bme280init(i2c)
    while True:
        temp, pressure, humid = readBME280()
        fbuff.fill(0)
        fbuff.fill(0)
        fbuff.text("BME280", 0, 0, 1)
        fbuff.text("tmp:", 0, 16, 1)
        fatntext("%.2f"%(temp), 32, 10)
        fbuff.text("prs:", 0, 36, 1)
        fatntext("%.2f"%(pressure), 32, 30)
        fbuff.text("hum:", 0, 54, 1)
        fatntext("%.2f"%(humid), 32, 50)
        oledshow()
        time.sleep(0.11)

if devname == "SHT31D":
    from SHT31D_funcs import SHT31Dsetup, readSHT31D
    SHT31Dsetup(i2c)
    while True:
        tmp, humid = readSHT31D()
        fbuff.fill(0)
        fbuff.text("SHT31-D", 0, 0, 1)
        fbuff.text("temperature:", 10, 10, 1)
        fatntext("%.2f"%tmp, 20, 22)
        fbuff.text("humidity:", 10, 40, 1)
        fatntext("%.2f"%humid, 20, 50)
        oledshow()
        time.sleep(0.1)
        
if devname == "MLX90621":
    fbuff.text("11", 50, 50, 1)
    oledshow()
    from MLX90621_funcs import measure, cellfillplot
    fbuff.text("22", 59, 50, 1)
    oledshow()
    while True:
        temperatures = measure()
        tmin = min(temperatures)
        tmax = max(tmin+0.001, max(temperatures))
        fbuff.fill(0)
        fbuff.text("MLX90621", 0, 0, 1)
        for i in range(64):
            cellfillplot(fbuff, i//4, 1+(3-(i%4)), ((temperatures[i] - tmin)/(tmax - tmin))**2)
        fbuff.text("min=%.3f"%tmin, 12, 48)
        fbuff.text("max=%.3f"%tmax, 12, 56)
        oledshow()

if devname == "PX4PITOT":
    while True:
        k = i2c.readfrom(0x28, 4)
        status = k[0] & 0xC0; 
        fbuff.fill(0)
        fbuff.text("PX4PITOT", 0, 0, 1)
        if status != 0xC0:
            rawpress = ((k[0] & 0x3F) << 8) | (k[1]) 
            rawtemp = (k[2] << 3) | (k[3] >> 5)
            psi = (rawpress)*(1.0/(0x3FFF*0.4)) - 1.25
            diffpressure = psi*6894.75728
            temp = (rawtemp)*(200.0/0x7FF) - 50
            #airspeed = math.sqrt(abs(2*getdiffpressure())/1.2);
            fbuff.text("diffpress:", 10, 10, 1)
            fatntext("%.2f"%diffpressure, 20, 22)
            fbuff.text("temp:", 10, 40, 1)
            fatntext("%.2f"%temp, 20, 50)
        else:
            fbuff.text("BAD", 20, 20, 1)
        oledshow()
        time.sleep(0.2)
        
if devname == "BNO055":
    from BNO055serial_funcs import InitBNO055, BNO055absorientation, BNO055calibstat
    InitBNO055(uart)
    while True:
        fbuff.fill(0)
        calibstat = None
        try:
            pitch, roll, orient = BNO055absorientation()
            calibstat = BNO055calibstat()
        except Exception as e:
            fbuff.text("BNO055", 0, 0, 1)
            fbuff.text(e.args[0], 0, 16, 1)
        if calibstat is not None:  
            fbuff.text("BNO055 :{0:08b}".format(calibstat), 0, 0, 1)
            fbuff.text("pitch:", 8, 16, 1)
            fatntext("%d"%int(pitch), 64, 10)
            fbuff.text("roll:", 16, 32, 1)
            fatntext("%d"%int(roll), 64, 28)
            fbuff.text("orient:", 0, 52, 1)
            fatntext("%d"%int(orient), 64, 48)
        oledshow()
        time.sleep(0.1)
        
if devname == "GPS":
    while True:
        x = uart.readline()
        if x is not None:
            fbuff.fill(0)
            fbuff.text("GPS serial", 0, 0, 1)
            x = x.strip()
            for i in range(len(x)//16 + 1):
                fbuff.text(x[i*16:i*16+16], 0, 15+i*8, 1)
            oledshow()
            time.sleep_ms(100)
        time.sleep_ms(50)


In [ ]:
        
# read any GPS serial signals on 
fbuff.text("Znone", 120, 56, 1)
oledshow()
time.sleep(2.0)

from OLED_grapher import scrollinit, addscrollgraph, plotscrollgraph
scrollinit()
a = machine.ADC(machine.Pin(36))  # Pin4 from top left
a.atten(a.ATTN_11DB)
a.width(a.WIDTH_12BIT)
while True:
    v = a.read()
    addscrollgraph(v, time.ticks_ms())
    plotscrollgraph(fbuff)
    fbuff.text("ADCPin36",0,0,1)
    fbuff.text(str(v),0,8,1)
    oledshow()
time.sleep(0.1)
    

In [5]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


1.883429e+11
array('f', [4.98294e-08, 5.413676e-08, 5.297261e-08, 4.843241e-08, 5.215771e-08, 5.565016e-08, 5.530091e-08, 5.099355e-08, 5.36711e-08, 5.681432e-08, 5.646507e-08, 5.215771e-08, 5.483525e-08, 5.937545e-08, 5.82113e-08, 5.413676e-08, 5.925904e-08, 6.193659e-08, 6.182017e-08, 5.553375e-08, 6.042319e-08, 6.391564e-08, 6.251866e-08, 5.704715e-08, 6.135451e-08, 6.449772e-08, 6.426489e-08, 5.727998e-08, 6.193659e-08, 6.589471e-08, 6.50798e-08, 5.786205e-08, 6.193659e-08, 6.426489e-08, 6.449772e-08, 5.739639e-08, 6.135451e-08, 6.531263e-08, 6.310074e-08, 5.5883e-08, 6.077244e-08, 6.35664e-08, 6.251866e-08, 5.541734e-08, 5.797847e-08, 6.053961e-08, 6.042319e-08, 5.215771e-08, 5.66979e-08, 5.762922e-08, 5.762922e-08, 5.076072e-08, 5.448601e-08, 5.681432e-08, 5.599941e-08, 4.761751e-08, 5.273978e-08, 5.28562e-08, 5.355469e-08, 4.656977e-08, 5.076072e-08, 5.13428e-08, 5.052789e-08, 4.249523e-08])


In [12]:
for i in range(64):
    # Section 7.3.3.1 
    VIR = ustruct.unpack("<h", z.irData[i*2:i*2+2])[0]  # <-- this is ridiculously slow
    VIRcompensated = VIR - (z.AI[i] + z.BI[i]*(z.Ta - 25))
    t = (VIRcompensated/z.alphacompI[i] + z.TaK4)**0.25 - 273.15
#    temperatures[i] = (VIRcompensated/alphacompI[i] + TaK4)**0.25 - 273.15
    print(t)


(232.1904+505.3405j)
255.2329
138.8689
617.7916
364.5426
(228.2065+501.3565j)
349.6132
204.2651
(171.2454+444.3954j)
(7.914703+281.0647j)
196.8909
678.6301
(162.585+435.735j)
385.3444
255.5977
(164.3778+437.5278j)
(230.6434+503.7934j)
624.9637
219.019
274.9163
335.3936
385.6419
188.2259
429.9454
627.5069
314.9575
118.1405
152.5858
388.193
588.1402
(-61.05881+212.0912j)
390.2385
529.601
384.9928
(102.9981+376.1481j)
513.6724
(164.6468+437.7968j)
232.7409
(89.06601+362.216j)
654.6141
631.3577
303.1949
646.406
(200.9364+474.0864j)
304.792
388.2053
352.3581
549.3299
380.3126
332.7258
(117.5154+390.6654j)
315.8437
(190.9999+464.1499j)
385.6763
385.8643
594.77
565.786
157.1682
(140.0693+413.2194j)
(86.72226+359.8723j)
291.1006
(99.40198+372.552j)
388.854
(57.10229+330.2523j)


In [17]:
print(z.AI)
print(z.BI)


array('i', [-105, -111, -101, -100, -109, -96, -106, -105, -102, -79, -95, -98, -86, -86, -90, -88, -95, -89, -93, -84, -75, -94, -82, -94, -74, -91, -92, -87, -82, -83, -81, -95, -79, -82, -69, -87, -78, -82, -71, -81, -75, -73, -85, -78, -74, -72, -70, -72, -72, -72, -79, -63, -66, -68, -68, -52, -71, -71, -72, -60, -80, -68, -76, -65])
array('f', [-0.734375, -0.6640625, -0.703125, -0.6640625, -0.703125, -0.703125, -0.703125, -0.734375, -0.6640625, -0.6328125, -0.6640625, -0.703125, -0.6328125, -0.6015625, -0.6328125, -0.6328125, -0.6640625, -0.5703125, -0.6015625, -0.6328125, -0.6328125, -0.6328125, -0.6015625, -0.6640625, -0.5703125, -0.6015625, -0.5703125, -0.6640625, -0.6015625, -0.6015625, -0.6015625, -0.6640625, -0.5703125, -0.53125, -0.5703125, -0.6328125, -0.5703125, -0.5703125, -0.5703125, -0.5703125, -0.53125, -0.53125, -0.53125, -0.5703125, -0.5703125, -0.5703125, -0.53125, -0.5703125, -0.5703125, -0.53125, -0.53125, -0.53125, -0.5703125, -0.5, -0.5, -0.53125, -0.5703125, 

In [9]:
a = machine.ADC(machine.Pin(36))  # Pin4 from top left
a.atten(a.ATTN_11DB)
a.width(a.WIDTH_12BIT)
scrollinit()
while True:
    v = a.read()
    addscrollgraph(v, time.ticks_ms())
    plotscrollgraph(fbuff)
    fbuff.text("ADCPin36",0,0,1)
    oledshow()


..........................................................
**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

